# Calibration

**scikit-rf** can be used to calibrate data taken from a VNA
* assuming you have measured an acceptable set of standards,
* and have a corresponding set ideal responses.

This may be referred to as *offline* calibration, because it is not occurring onboard the VNA itself. 

This technique provides maximum flexibility for non-conventional calibrations, and preserves all raw data.

There are many calibration algorithms implemented in scikit-rf. The choice between them depends of the media, the available calibration standards and the desired trade-off between accuracy and effort. 

The calibration algorithms available in scikit-rf are listed in the [calibration API's page](https://scikit-rf.readthedocs.io/en/latest/api/calibration/index.html).

## Creating a Calibration

Calibrations are performed through a `Calibration`. In General, `Calibration` objects require two arguments:

*  a list of measured `Network`'s
*  a list of ideal `Network`'s

The `Network` elements in each list: 
* must all be similar (same number of ports, frequency info, etc)
* must be aligned to each other: the first element of ideals list must correspond to the first element of measured list.

*Self-calibration* algorithms, such as TRL, do not require predefined ideal responses.

## One-Port Example

To construct a one-port calibration, you need to
* have measured at least three standards
* have their known *ideal* responses in the form of `Network`s. 

These `Network` can be created from touchstone files or from ideal analytic's reponse.

In the following script  the measured and ideal touchstone files for a conventional short-open-load (SOL) calkit are in folders `measured/` and `ideal/`, respectively. These are used to create a `OnePort` Calibration and correct a measured DUT 

	# a list of Network types, holding 'ideal' responses
	my_ideals = [\
	        rf.Network('ideal/short.s1p'),
	        rf.Network('ideal/open.s1p'),
	        rf.Network('ideal/load.s1p'),
	        ]
	
	# a list of Network types, holding 'measured' responses
	my_measured = [\
	        rf.Network('measured/short.s1p'),
	        rf.Network('measured/open.s1p'),
	        rf.Network('measured/load.s1p'),
	        ]

Create a Calibration instance:   

	cal = rf.calibration.OnePort(\
	        ideals = my_ideals,
	        measured = my_measured,
	        )

Run, and apply calibration to a DUT:

	# run calibration algorithm
	cal.run() 
	
	# apply it to a dut
	dut = rf.Network('my_dut.s1p')
	dut_caled = cal.apply_cal(dut)
	dut_caled.name =  dut.name + ' corrected'

## Concise One-port
This example achieves the same task as the one above except in a more concise *programmatic* way. 

    my_ideals = rf.load_all_touchstones_in_dir('ideals/')
    my_measured = rf.load_all_touchstones_in_dir('measured/')
    
    
    ## create a Calibration instance
    cal = rf.calibration.OnePort(\
	    ideals = [my_ideals[k] for k in ['short','open','load']],
	    measured = [my_measured[k] for k in ['short','open','load']],
	    )

##  Two-port Calibrations
**skrf** supports a few different two-port algorithms: 

* Algorythms bases on 12-term error model, such as the traditional `SOLT`. Similar to the OnePort example.
* The `EightTerm` calibrations can  be constructed from any number of standards, providing that:
 * you need 3 **two-port** standards:
   * one must be transmissive, 
   * one must provide a known impedance 
   * one must be reflective.
 * Important detail:  of using the 8-term error model formulation is that switch-terms may need to be measured in order to achieve a high quality calibration
 
Note, that the word 8-term is used in the literature to describe a specific error model used by a variety of calibration algorithms, like TRL, LRM, etc. The EightTerm class, is an implementation of the algorithm cited above, which does not use any self-calibration. (---------NOT CLEAR--------)

## Using one-port ideals in two-port Calibration

Commonly, you have data for ideal data for reflective standards in the form of one-port touchstone files (ie `.s1p`).

To use this with skrf's two-port calibration method you need to create a two-port network that is a composite of the two networks. The  function `skrf.network.two_port_reflect` does this:

<img src="figures/2_1port_to_1_2port.svg"/>

    short = rf.Network('ideals/short.s1p')
    shorts = rf.two_port_reflect(short, short)

## Switch-terms

Originally described by Roger Marks, *switch-terms* account for the fact that the 8-term (aka *error-box* ) model is overly simplified:
* The two error networks change slightly depending on which port is being excited due to internal switch within the VNA. 
* Switch terms describe the imperfect load when the source is switch to the other port.

Switch terms can be directly measured with a low insertion loss transmissive standard (like a thru) connected between the cables connected to the instrument ports. 

Sometime, a custom measurement configuration is available on the VNA itself. 

Without switch-term measurements, your calibration quality will vary depending on properties of you VNA.

## SOLT Example

Two-port calibration is accomplished in an identical way to one-port, except all the standards are two-port networks. 

This is even true of reflective standards (S21=S12=0). 

So if you measure reflective standards you must measure two of them simultaneously, and store information in a two-port:

<img src="figures/VNA_2_1port.svg"/>

For example, connect a short to port-1 and a load to port-2, and save a two-port measurement as 'short,load.s2p' or similar
   
    # a list of Network types, holding 'ideal' responses
    my_ideals = [
	    rf.Network('ideal/thru.s2p'),
	    rf.Network('ideal/short, short.s2p'),
	    rf.Network('ideal/open, open.s2p'),
	    rf.Network('ideal/load, load.s2p'),
	    ]
    
    # a list of Network types, holding 'measured' responses
    my_measured = [
	    rf.Network('measured/thru.s2p'),
	    rf.Network('measured/short, short.s2p'),
	    rf.Network('measured/open, open.s2p'),
	    rf.Network('measured/load, load.s2p'),
	    ]

    ## create a SOLT instance
    cal = rf.calibration.SOLT(
	    ideals = my_ideals,
	    measured = my_measured,
	    )

    # run calibration algorithm
    cal.run() 
    
    # apply it to a dut
    dut = rf.Network('my_dut.s2p')
    dut_caled = cal.apply_cal(dut)

# Example : one port tiered calibration?